In [1]:
import numpy as np
from scipy.special import logsumexp
import nbimporter
from hmm import GaussianHMM


In [2]:
class logreg:
    """uses Newton's Method of optimisation to find logistic regression parameters of best fit given data and responsibilites of each data pt (loggamma)"""

    def __init__(self, X, ys, gamma):

        self.n = len(X)

        self.gamma = gamma.reshape((self.n, 1))

        self.xmat = np.hstack((np.ones((self.n, 1)), X.reshape((self.n, 1))))

        self.ys = ys.reshape((self.n, 1))

        self.betas = np.array([0.1, 0.2])



    def logit(self):
        xBT = ((self.gamma * self.xmat) @ self.betas.reshape((2,1)))
        return xBT.reshape((self.n, 1))


    def logistic(self):
        xBT = self.logit()
        return (1.0 / (1.0 + np.exp(-xBT))).reshape((self.n, 1))


    def ll(self):
        P = self.logistic()
        return self.ys * np.log(P) + (1 - self.ys) * np.log(1 - P)

    def grad(self, P):
        grad = self.xmat.T @ (self.ys - P)
        return grad


    def hess(self, P):
        W = np.diag((P * (1 - P)).flatten())
        H = (-1 * self.xmat.T) @ W @ self.xmat
        return H


    def newtons_method(self, atol):

        ll = float('-inf')
        llpts = []
        i = 0

        for i in range(1000):

            i+= 1

            P = self.logistic()
            g = self.grad(P)
            h = self.hess(P)

            self.betas -= (np.linalg.pinv(h) @ g).reshape(2,)

            ll_new = logsumexp(self.ll())

            if (ll_new - ll) < 0:
                raise Exception("LL can't decrease")
            if (ll_new-ll) < atol:
                break

            ll = ll_new
            llpts.append(ll)

        return self.betas, llpts

In [3]:

class GLMHMM:
    """
    given a set of observations and number of states, this class uses the expectation-maximisation algorithm to fit an HMM model,
    finding the time-dependent transition probabilities between the states generating the observations, as well as the logistic
    regression parameters of best fit for each state
    """

    def __init__(self, k, X, ys, pi = None, logpi = None, A = None, logA = None, logreg_models = None, ll_pts = None):
        """initalises the parameters of the HMM, requires K to be initalised by user"""

        # K - number of states
        self.k = k

        self.X = X
        self.ys = ys

        # logistic regression models for each state
        if logreg_models is None:
            self.logreg_models = [logreg(X, ys, np.ones(X.shape[0])) for k in range(self.k)]

        # list of LLs to update at each iteration
        self.ll_pts = []

        # initial probabilities
        self.pi = [0.3, 0.3, 0.4]
        self.logpi = np.log(self.pi)

        # transition matrix
        self.A = [[0.7, 0.2, 0.1],
                [0.2, 0.7, 0.1],
                [0.1, 0.2, 0.7]]
        self.logA = np.log(self.A)

        #emission distributions
        self.ll_lists = [self.logreg_models[k].ll().flatten().tolist() for k in range(self.k)]

        #Yahoo! initialised


    def calc_logalpha(self):
        """calculates alpha of the forward-backward algorithm, E-step"""
        t = len(self.X)
        logalpha = np.zeros((t, self.k))

        for k in range(self.k):
            ll_k = self.ll_lists[k]
            logalpha[0, k] = self.logpi[k] + ll_k[0]

        for t in range(1, t):
            for k in range(self.k):
                ll_k = self.ll_lists[k]
                logalpha_temp = []

                for j in range(self.k):

                    logalpha_temp.append( logalpha[t-1, j] + self.logA[j, k]  )

                logalpha_sum = logsumexp(logalpha_temp)
                logalpha[t, k] = logalpha_sum + ll_k[t]

        return logalpha


    def calc_logbeta(self):
        """calculates beta of the forward-backward algorithm, E-step"""
        T = len(self.X)
        logbeta = np.zeros((T, self.k))
        logbeta[T-1, :] = 0

        for t in range(T-2, -1, -1):

            for k in range(self.k):
                logbeta_temp = []

                for j in range(self.k):
                    ll_j = self.ll_lists[j]
                    logbeta_temp.append(logbeta[t+1, j] + self.logA[k, j] + ll_j[t+1] )

                logbeta[t, k] = logsumexp(logbeta_temp)


        return logbeta


    def calc_loggamma(self, logalpha, logbeta):
        """calculates gamma, the expectation of the latent - the marginal posterior distribution, E-step"""
        T = len(self.X)
        loggamma = np.zeros((T, self.k))

        for t in range(T):
            gamma = logalpha[t, :] + logbeta[t, :]
            gamma_denom = logsumexp(gamma)
            gamma -= gamma_denom
            loggamma[t, :] = gamma

        return loggamma


    def calc_logxi(self, logalpha, logbeta):
        """calculates xi, the expectation of the latent transitions - the joint posterior distribution, E-step"""
        T = len(self.X)
        logxi = np.zeros((T-1, self.k, self.k))

        for t in range(T-1):

            logxi_temp = np.zeros((self.k, self.k))

            for k in range(self.k):
                for j in range(self.k):

                    ll_j = self.ll_lists[j]
                    logxi_temp[k, j] = logalpha[t, k] + self.logA[k, j] + ll_j[t+1] + logbeta[t+1, j]

            # normalisation step
            logxi[t, :, :] = logxi_temp - logsumexp(logxi_temp)

        return logxi


    def up_pi(self, loggamma):
        """updates pi parameter in M-step"""
        loggamma_col_sum = loggamma[0, :]

        self.pi = np.exp(loggamma_col_sum)
        self.logpi = loggamma_col_sum
        return


    def up_A(self, logxi, loggamma):
        """updates A parameter in M-step"""
        temporary_logA = np.zeros((self.k, self.k))

        for i in range(self.k):
            for j in range(self.k):
                temporary_logA[i, j] = (logsumexp(logxi[:, i, j]) - logsumexp(loggamma[:-1, i]))

        self.logA = temporary_logA
        self.A = np.exp(self.logA)
        return


    def up_logreg_models(self, loggamma):
        """updates logistic regressions for each state in M-step"""
        gamma = np.exp(loggamma)
        self.logreg_models = [logreg(self.X, self.ys, gamma[:, k]) for k in range(self.k)]
        return

    def up_ll_list(self):
        """updates emission distributions in M-step"""
        self.ll_lists = [self.logreg_models[k].ll().flatten().tolist() for k in range(self.k)]
        return


    def EM(self):
        """Expectation Maximisation Algorithm, takes expectation of the latent, then uses the argmax of Q-function to update parameter of model"""
        #Expectation Step
        logalpha = self.calc_logalpha()
        logbeta = self.calc_logbeta()
        loggamma = self.calc_loggamma(logalpha, logbeta)
        logxi = self.calc_logxi(logalpha, logbeta)

        #Maximisation Step
        self.up_pi(loggamma)
        self.up_A(logxi, loggamma)
        self.up_logreg_models(loggamma)
        self.up_ll_list()
        return


    def calc_ll(self, logalpha):
        """calculates the log-likelihood of each iteration to track fit progress"""
        ll = (logsumexp(logalpha[-1, :]))

        self.ll_pts.append(ll)

        return ll


    def fit(self, max_iter, a_tol):
        """given data - observations, max iteration - the max iteration of EM alg, a_tol - smallest step of conversion, fits HMM"""
        ll = float('-inf')

        for i in range(max_iter):

            self.EM()
            logalpha = self.calc_logalpha()
            ll_new = self.calc_ll(logalpha)


            if (ll_new - ll) < a_tol:
                break


            ll = ll_new

        print(f"LL: {self.ll_pts}")

        return[model.betas for model in self.logreg_models], self.ll_pts, self.A, all(self.ll_pts[i] > self.ll_pts[i - 1] for i in range(1, len(self.ll_pts)))